Origin-destination matrix estimation

Possibly useful for constructing P matrix: probability of going through edges when moving from A to B

- https://openrouteservice.org/services/  (e.g. nearest)
- Google Maps API, maybe Google Distance Matrix API, maybe Google Geotagging API
- https://www.opentripplanner.org/ (Helps: https://github.com/jeromemayaud/OpenTripPlanner-OD-matrix)
Directions API: Directions between multiple locations, Distance Matrix API: Travel times and distances between locations<br>
Routes API: Performance optimized versions of the Directions and Distance Matrix APIs<br>
Roads API: Snap-to-road functionality to accurately trace GPS breadcrumbs.  maybe Places API

"QNEAT3 OD Matrix Calculation": https://github.com/kahlulz/Olongapo-City-Hospital-Network-Analysis<br>
Spatial Analysis: QGIS is utilized to conduct spatial analysis. Hospital locations were mapped, and distance-based calculations were performed using QNEAT3 OD Matrix Calculation. <br>
https://gis.stackexchange.com/questions/403556/what-unit-is-represented-by-od-matix-costs-in-qneat3-within-qgis-3-6-noosa?newreg=400dd6f344ab480a959ecfc8002019b9 <br>
https://root676.github.io/OdMatrixAlgs.html <br>
https://www.qgistutorials.com/en/docs/3/origin_destination_matrix.html

In [ ]:
import map_functions

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

gdf = gpd.read_file("data/sources/Hungary_2022_static_road_data.geojson")

In [ ]:
import geopandas as gpd
import pandas as pd

gdf = gpd.read_file("data/Hungary_2022_static_road_data.geojson")

### Combining roads: based on kszam (ID)

Functions included in `map_functions.py`

In [ ]:
gdf_new = map_functions.combine_roads_total_simple(gdf)

In [ ]:
gdf_new[~gdf_new['min_5_traffic'].isna()]

,kszam,min_traffic,min_5_traffic,max_traffic,max_5_traffic,avg_traffic,geometry,data_df,data_json
0,M43,20728,20728,37030,34512,27256.641509,"MULTILINESTRING ((731774.360 106214.871, 73205...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
1,M4,3718,3718,57443,39233,20643.589744,"MULTILINESTRING ((848187.895 212117.248, 84819...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
2,M35,10875,10875,19402,18800,15349.763636,"MULTILINESTRING ((835825.477 255049.021, 83584...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
3,M15,21871,26100,28451,28451,26436.523810,"MULTILINESTRING ((510618.213 297665.117, 51055...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
4,M44,8115,8851,9583,9583,9211.729167,"MULTILINESTRING ((800011.635 153748.361, 80293...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
...,...,...,...,...,...,...,...,...,...
3341,M8,7144,7144,13185,13185,10596.000000,"MULTILINESTRING ((637084.268 175766.926, 63722...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
3342,M80,4712,5128,5828,5494,5358.944444,"MULTILINESTRING ((464853.113 189984.635, 46479...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
3343,M9,4849,4849,4849,4849,4849.000000,"MULTILINESTRING ((626831.533 115238.519, 62690...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
3344,2,5823,8282,35266,35266,17420.816667,"MULTILINESTRING ((656911.982 269457.874, 65690...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."


## P Matrix: paths with Google

In [ ]:
import json
key = json.loads(open('keys.json').read())[""]

In [ ]:
import googlemaps

gmaps = googlemaps.Client(key=key)
directions_result = gmaps.directions("Hegyeshalom", "Mosonszolnok", alternatives=True)
#Possible directions test
for i, route in enumerate(directions_result):
    print(f"Route {i+1}:")
    for step in route['legs'][0]['steps']:  
        print(step['html_instructions'])
    print("\n\n")

Route 1:
Head <b>southwest</b> on <b>Alkotmány u.</b>
Turn <b>left</b>
Turn <b>right</b> at <b>Orgona u.</b>
Turn <b>right</b>
Turn <b>left</b>
Continue onto <b>Szabadság út</b>
Turn <b>right</b> onto <b>Jókai u.</b><div style="font-size:0.9em">Destination will be on the right</div>



Route 2:
Head <b>northeast</b> on <b>Alkotmány u.</b> toward <b>Damjanich u.</b>
Turn <b>right</b> onto <b>Damjanich u.</b>
Turn <b>left</b> onto <b>Szolnoki u.</b>
Continue onto <b>Kertalja u.</b>
Turn <b>right</b> onto <b>Fő u.</b>/<wbr/><b>Route 1</b><div style="font-size:0.9em">Continue to follow Route 1</div>
At the roundabout, take the <b>3rd</b> exit and stay on <b>Route 1</b>
At the roundabout, take the <b>2nd</b> exit and stay on <b>Route 1</b>
Turn <b>right</b> onto <b>Szolnoki út</b>
Continue onto <b>Óvári u.</b>
Continue onto <b>Fő u.</b>
Turn <b>left</b> onto <b>Szabadság út</b>
Turn <b>right</b> onto <b>Jókai u.</b><div style="font-size:0.9em">Destination will be on the right</div>



Route